# Aplicación Multi-Agente Básica con CrewAI: Sistema multi-agente para reemplazar el equipo de marketing de contenidos

## ¿Qué vamos a construir?
En este notebook aprenderemos a crear una aplicación que utiliza múltiples agentes de IA trabajando en colaboración. Estos agentes simularán un equipo completo de marketing de contenidos:
- Un investigador que busca información en internet
- Un gestor de blog que escribe artículos
- Un gestor de redes sociales que crea tweets
- Un director de marketing que supervisa todo el contenido

## Crear la Aplicación
* **Paso 1: Entorno Virtual** - Siempre recomendamos crear un entorno virtual primero. Usamos pyenv para hacerlo, pero puedes usar virtualenv o conda.
* **Paso 2: Comandos de instalación** - Una vez que tengas el entorno virtual configurado, ejecuta los siguientes comandos en la terminal:

In [ ]:
# Comando 1: Crear un nuevo proyecto con Poetry (gestor de dependencias de Python)
# Poetry nos ayuda a manejar las librerías y dependencias de forma organizada
#poetry new basiccrewai --name app

In [ ]:
# Comando 2: Navegar al directorio del proyecto recién creado
#cd basiccrewai

In [ ]:
# Comando 3: Instalar las dependencias necesarias
# - crewai: Framework para crear equipos de agentes de IA
# - beautifulsoup4: Librería para extraer datos de páginas web (web scraping)
#poetry add crewai beautifulsoup4

## Solución a un Error Común
* **Problema**: Al ejecutar el comando anterior, puede aparecer un error relacionado con la versión de Python
* **Solución**: El mensaje de error es auto-explicativo. Para resolver el problema:
    1. Abre el archivo `pyproject.toml` de la aplicación
    2. Busca la línea 9 y actualízala a:
    * `python = ">=3.12,<=3.13"` 
* Esto indica que el proyecto requiere Python versión 3.12 o 3.13

## Abrir el Proyecto en el Editor y Crear el Archivo Principal
* **Editor recomendado**: Visual Studio Code (VSCode) - es gratuito y muy potente
* **Archivo principal**: Crearemos el archivo `app/multiagent.py` que contendrá todo nuestro código
* Este archivo será el cerebro de nuestra aplicación multi-agente

## Configurar las Claves API (archivo .env)
Ahora crearemos un archivo `.env` en el directorio raíz con las credenciales necesarias. 
**¡IMPORTANTE!** Este archivo contiene información confidencial y NO debe compartirse públicamente.

### Claves necesarias:

**1. OpenAI API Key** - Para usar los modelos de lenguaje GPT
```
OPENAI_API_KEY=tu_clave_aqui
```

**2. LangSmith** - Para monitorear y rastrear el comportamiento de los agentes
```
LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=tu_clave_aqui
LANGCHAIN_PROJECT=nombre_de_tu_proyecto
```

**3. Tavily API Key** - Para realizar búsquedas en internet
```
TAVILY_API_KEY=tu_clave_aqui
```

## Seguridad: Proteger tus Claves API
* **Nombre del proyecto LangSmith**: Lo llamaremos "basiccrewai" para identificar fácilmente las métricas
* **Archivo .gitignore**: Crea este archivo y añade `.env` dentro para evitar que las claves se suban a GitHub
  - Esto es CRUCIAL para mantener tus credenciales privadas y seguras
  - Si subes tus claves a GitHub, personas malintencionadas podrían usarlas y gastarte dinero

## Importar los Módulos Necesarios en app/multiagent.py
En esta sección importaremos todas las librerías que necesitaremos para crear nuestros agentes de IA.
Cada librería tiene una función específica que veremos a continuación.

## Instalación de Dependencias Adicionales
* **Paso previo**: Antes de importar, debemos instalar las siguientes librerías mediante terminal:
  - `python-dotenv`: Para leer las variables de entorno del archivo .env
  - `typing`: Para añadir tipado de datos (hace el código más robusto)

In [ ]:
# Comando para instalar python-dotenv y typing
# Ejecutar este comando en la terminal:
#!pip install python-dotenv typing

## Importar los Módulos en app/multiagent.py
Ahora sí, importemos todas las librerías necesarias. Cada import tiene un propósito específico que explicaremos:

In [ ]:
# ========================================
# SECCIÓN 1: IMPORTACIÓN DE LIBRERÍAS
# ========================================

# import requests  # Para hacer peticiones HTTP a páginas web
# import os  # Para interactuar con el sistema operativo
# from dotenv import load_dotenv  # Para cargar las variables de entorno desde el archivo .env
# from bs4 import BeautifulSoup  # Para analizar y extraer datos de HTML (web scraping)
# from crewai import Agent, Task, Crew  # Los componentes principales de CrewAI para crear agentes
# from langchain.tools import tool  # Decorador para crear herramientas personalizadas
# from langchain_openai import ChatOpenAI  # Para conectar con los modelos de OpenAI (GPT)
# from langchain_community.tools.tavily_search import TavilySearchResults  # Para búsquedas en internet

# load_dotenv()  # Carga todas las variables del archivo .env (API keys, etc.)

## Seleccionar el Modelo de Lenguaje (LLM)
Aquí definimos qué modelo de IA usarán nuestros agentes para "pensar" y generar respuestas.
En este caso usamos **GPT-4 Turbo**, que es uno de los modelos más potentes de OpenAI.

In [ ]:
# ========================================
# SECCIÓN 2: CONFIGURACIÓN DEL MODELO LLM
# ========================================

# llm = ChatOpenAI(model="gpt-4-turbo-preview")  
# Creamos una instancia del modelo GPT-4 Turbo
# Este será el "cerebro" de todos nuestros agentes
# NOTA: GPT-4 es más caro que GPT-3.5, pero produce resultados de mayor calidad

## Definir las Herramientas que Usarán los Agentes
Los agentes necesitan "herramientas" para realizar acciones. Aquí definimos dos herramientas clave:

1. **Búsqueda en Internet con Tavily** - Permite a los agentes buscar información actualizada en la web
2. **Análisis HTML con BeautifulSoup** - Permite extraer y procesar el contenido de páginas web

### Parámetro importante: `return_direct=False`
* **Significado**: La herramienta devuelve resultados PRIVADAMENTE al agente, no directamente a la aplicación
* **Por qué es importante**: Permite que el agente procese la información antes de presentarla al usuario

In [ ]:
# ========================================
# SECCIÓN 3: DEFINICIÓN DE HERRAMIENTAS
# ========================================

# Herramienta 1: Procesador de contenido web
# @tool("process_search_tool", return_direct=False)  # El decorador @tool convierte la función en una herramienta usable por los agentes
# def process_search_tool(url: str) -> str:
#     """Herramienta para procesar contenido encontrado en internet."""
#     # Hace una petición HTTP a la URL proporcionada
#     response = requests.get(url=url)
#     # BeautifulSoup analiza el HTML de la página
#     soup = BeautifulSoup(response.content, "html.parser")
#     # Extrae solo el texto, eliminando todas las etiquetas HTML
#     return soup.get_text()

# Herramienta 2: Lista de todas las herramientas disponibles
# tools = [
#     TavilySearchResults(max_results=1),  # Búsqueda en internet (limitada a 1 resultado)
#     process_search_tool  # Procesador de contenido web que acabamos de crear
# ]

## Nota Importante sobre Costos y Limitaciones

### ⚠️ Limitación de Resultados de Búsqueda
* **Configuración actual**: `max_results=1` - Solo buscamos 1 resultado en internet
* **Por qué**: Para fines demostrativos y para controlar costos
* **Impacto**: Esto LIMITA la calidad de la búsqueda, ya que el agente solo analiza un resultado en lugar de comparar múltiples fuentes

### 💰 Recomendación para Producción
* En aplicaciones profesionales, aumenta `max_results` a un número mayor (ej: 5-10)
* **Considera los costos**: Más resultados = más tokens = más dinero
* Encuentra el balance entre calidad de resultados y presupuesto

## Crear el Agente: Investigador Online (online_researcher)
Este es nuestro primer agente. Su trabajo es buscar información en internet, como un periodista investigando para escribir un artículo.

In [ ]:
# ========================================
# SECCIÓN 4: AGENTE 1 - INVESTIGADOR ONLINE
# ========================================

# online_researcher = Agent(
#     # ROL: Define la identidad del agente
#     role="Investigador Online",
#     
#     # OBJETIVO: Define qué debe lograr el agente
#     goal="Investigar el tema en internet",
#     
#     # BACKSTORY: La "historia personal" del agente - define su personalidad y capacidades
#     backstory="""Tu rol principal es funcionar como un asistente inteligente de investigación online, 
#     experto en explorar internet para encontrar las historias más recientes y relevantes en diversos 
#     sectores como política, tecnología, salud, cultura y eventos globales. Tienes la capacidad de 
#     acceder a una amplia gama de fuentes de noticias online, blogs y plataformas de redes sociales 
#     para recopilar información en tiempo real.""",
#     
#     # VERBOSE: Si es True, muestra el proceso de pensamiento del agente (útil para debugging)
#     verbose=True,
#     
#     # ALLOW_DELEGATION: Si es True, el agente puede delegar tareas a otros agentes
#     allow_delegation=True,
#     
#     # TOOLS: Lista de herramientas que puede usar este agente
#     tools=tools,
#     
#     # LLM: El modelo de lenguaje que usará para "pensar"
#     llm=llm
# )

## Crear el Agente: Gestor de Blog (blog_manager)
Este agente transforma la investigación en artículos de blog bien escritos y optimizados para SEO.

In [ ]:
# ========================================
# SECCIÓN 5: AGENTE 2 - GESTOR DE BLOG
# ========================================

# blog_manager = Agent(
#     role="Gestor de Blog",
#     goal="Escribir el artículo de blog",
#     
#     # El backstory define las responsabilidades específicas del gestor de blog
#     backstory="""Eres un Gestor de Blog. El rol de un Gestor de Blog abarca varias responsabilidades 
#     críticas destinadas a transformar borradores iniciales en artículos de blog pulidos y optimizados 
#     para SEO que atraigan y hagan crecer la audiencia. Comenzando con borradores proporcionados por 
#     investigadores online, el Gestor de Blog debe comprender a fondo el contenido, asegurándose de 
#     que se alinee con el tono del blog, la audiencia objetivo y los objetivos temáticos.
#     
#     Responsabilidades clave:
#     
#     1. Mejora de Contenido: Elevar la calidad del borrador mejorando la claridad, el flujo y el 
#        compromiso. Esto implica refinar la narrativa, añadir encabezados atractivos y asegurar 
#        que el artículo sea fácil de leer e informativo.
#     
#     2. Optimización SEO: Implementar mejores prácticas para la optimización de motores de búsqueda. 
#        Esto incluye investigación e integración de palabras clave, optimización de meta descripciones 
#        y asegurar que las estructuras de URL y etiquetas de encabezado mejoren la visibilidad en 
#        los resultados de búsqueda.
#     
#     3. Cumplimiento y Mejores Prácticas: Asegurar que el contenido cumpla con estándares legales 
#        y éticos, incluyendo leyes de derechos de autor y veracidad en la publicidad.
#     
#     4. Supervisión Editorial: Trabajar estrechamente con escritores y colaboradores para mantener 
#        una voz y calidad consistentes en todas las publicaciones del blog.
#     
#     5. Integración de Análisis y Retroalimentación: Revisar regularmente las métricas de rendimiento 
#        para entender el compromiso y las preferencias de la audiencia.
#     
#     En resumen, el Gestor de Blog juega un rol fundamental al conectar la investigación inicial con 
#     la publicación final, mejorando la calidad del contenido, asegurando compatibilidad SEO y 
#     alineándose con los objetivos estratégicos del blog.""",
#     
#     verbose=True,
#     allow_delegation=True,
#     tools=tools,
#     llm=llm
# )

## Crear el Agente: Gestor de Redes Sociales (social_media_manager)
Este agente convierte la investigación en tweets concisos y atractivos para Twitter (ahora X).

In [ ]:
# ========================================
# SECCIÓN 6: AGENTE 3 - GESTOR DE REDES SOCIALES
# ========================================

# social_media_manager = Agent(
#     role="Gestor de Redes Sociales",
#     goal="Escribir un tweet",
#     
#     # Este agente se especializa en contenido breve y atractivo para redes sociales
#     backstory="""Eres un Gestor de Redes Sociales. El rol de un Gestor de Redes Sociales, 
#     particularmente para gestionar contenido de Twitter, implica transformar borradores de 
#     investigación en tweets concisos y atractivos que resuenen con la audiencia y se adhieran 
#     a las mejores prácticas de la plataforma. Al recibir un borrador de un investigador online, 
#     el Gestor de Redes Sociales tiene varias funciones críticas:
#     
#     1. Condensación de Contenido: Destilar el mensaje principal del borrador en un tweet, que 
#        típicamente permite solo 280 caracteres. Esto requiere un enfoque agudo en la brevedad 
#        mientras se mantiene la esencia y el impacto del mensaje.
#     
#     2. Optimización del Compromiso: Crear tweets para maximizar el compromiso. Esto incluye el 
#        uso estratégico de lenguaje atractivo, hashtags relevantes y temas oportunos que resuenen 
#        con la audiencia objetivo.
#     
#     3. Cumplimiento y Mejores Prácticas: Asegurar que los tweets sigan las directrices de Twitter 
#        y las mejores prácticas, incluyendo el uso apropiado de menciones, hashtags y enlaces. 
#        También, adherirse a estándares éticos, evitando la desinformación y respetando las 
#        normas de derechos de autor.
#     
#     En resumen, el rol del Gestor de Redes Sociales es crucial para aprovechar Twitter para 
#     difundir información de manera efectiva, interactuar con seguidores y construir la presencia 
#     de la marca online. Esta posición combina habilidades creativas de comunicación con 
#     planificación estratégica y análisis para optimizar el impacto en redes sociales.""",
#     
#     verbose=True,
#     allow_delegation=True,
#     tools=tools,
#     llm=llm
# )

## Crear el Agente: Director de Marketing de Contenidos (content_marketing_manager)
Este es el agente supervisor que revisa y aprueba todo el contenido antes de su publicación.

### ⚠️ Parámetros importantes:
* **verbose=True**: Muestra el proceso de pensamiento (útil para ver cómo toma decisiones)
* **allow_delegation=True**: Puede delegar trabajo a otros agentes del equipo

In [ ]:
# ========================================
# SECCIÓN 7: AGENTE 4 - DIRECTOR DE MARKETING DE CONTENIDOS
# ========================================

# content_marketing_manager = Agent(
#     role="Director de Marketing de Contenidos",
#     goal="Gestionar el Equipo de Marketing de Contenidos",
#     
#     # Este agente actúa como supervisor y controlador de calidad
#     backstory="""Eres un excelente Director de Marketing de Contenidos. Tu rol principal es 
#     supervisar cada publicación del 'gestor de blog' y los tweets escritos por el 'gestor de 
#     redes sociales' y aprobar el trabajo para su publicación. Examina el trabajo y regula el 
#     lenguaje violento, contenido abusivo y contenido racista.
#     
#     Capacidades:
#     
#     Revisión Editorial: Analizar los borradores finales del gestor de blog y del gestor de 
#     redes sociales para verificar consistencia de estilo, alineación temática y flujo narrativo 
#     general.
#     
#     Aseguramiento de Calidad: Realizar verificaciones detalladas de precisión gramatical, 
#     corrección factual y adherencia a estándares periodísticos en el contenido de noticias, 
#     así como creatividad y efectividad en las publicaciones.
#     
#     Ciclo de Retroalimentación: Proporcionar retroalimentación constructiva tanto al gestor 
#     de blog como al gestor de redes sociales, facilitando un ambiente colaborativo para la 
#     mejora continua en la creación y presentación de contenido.""",
#     
#     verbose=True,  # Importante: muestra el razonamiento del supervisor
#     allow_delegation=True,  # Puede pedir a otros agentes que rehagan su trabajo
#     tools=tools,
#     llm=llm
# )

## Definir las Tareas para Cada Agente
Ahora que tenemos los agentes, necesitamos decirles exactamente qué hacer. Cada tarea tiene:
* **description**: Qué debe hacer el agente
* **expected_output**: Qué tipo de resultado esperamos
* **agent**: Qué agente es responsable de esta tarea

In [ ]:
# ========================================
# SECCIÓN 8: DEFINICIÓN DE TAREAS
# ========================================

# TAREA 1: Investigación Online
# task1 = Task(
#     # Descripción detallada de lo que debe hacer el agente
#     description="""Escríbeme un informe sobre Comportamiento Agéntico (Agentic Behavior). 
#     Después de la investigación sobre Comportamiento Agéntico, pasa los hallazgos al gestor 
#     de blog para generar el artículo de blog final. Una vez hecho, pásalo al gestor de redes 
#     sociales para escribir un tweet sobre el tema.""",
#     
#     # Qué esperamos recibir como resultado
#     expected_output="Informe sobre Comportamiento Agéntico",
#     
#     # Quién es responsable de esta tarea
#     agent=online_researcher
# )

In [ ]:
# TAREA 2: Escribir Artículo de Blog
# task2 = Task(
#     description="""Usando los hallazgos de la investigación del corresponsal de noticias, 
#     escribe un artículo para el blog. La publicación debe contener enlaces a las fuentes 
#     citadas por el investigador online. Tu respuesta final DEBE ser el artículo de blog 
#     completo de al menos 3 párrafos.""",
#     
#     # Este agente debe producir un artículo completo y bien estructurado
#     expected_output="Artículo de Blog",    
#     
#     # El gestor de blog es responsable de esta tarea
#     agent=blog_manager
# )

In [ ]:
# TAREA 3: Crear Tweet
# task3 = Task(
#     description="""Usando los hallazgos de la investigación del corresponsal de noticias, 
#     escribe un tweet. Tu respuesta final DEBE ser el tweet completo.""",
#     
#     # El tweet debe ser conciso (máximo 280 caracteres) y atractivo
#     expected_output="Tweet",
#     
#     # El gestor de redes sociales maneja esta tarea
#     agent=social_media_manager
# )

In [ ]:
# TAREA 4: Revisión y Aprobación Final
# task4 = Task(
#     description="""Revisar y armonizar meticulosamente el resultado final tanto del gestor 
#     de blog como del gestor de redes sociales, asegurando cohesión y excelencia en la 
#     publicación final. Una vez hecho, publica el informe final.""",
#     
#     # El director debe producir un informe final aprobado
#     expected_output="Informe Final",
#     
#     # El director de marketing de contenidos supervisa esta última tarea
#     agent=content_marketing_manager
# )

## Definir el Equipo (Crew) de Agentes
Ahora juntamos todos los agentes y tareas en un "Crew" (equipo).

### Proceso Secuencial
* Los agentes trabajarán uno tras otro (no simultáneamente)
* El orden de las tareas es importante: primero investiga, luego escribe, luego revisa

In [ ]:
# ========================================
# SECCIÓN 9: CREAR EL EQUIPO Y EJECUTAR
# ========================================

# Paso 1: Crear una lista con todos los agentes
# agents = [online_researcher, blog_manager, social_media_manager, content_marketing_manager]

# Paso 2: Crear el Crew (equipo) con todos los agentes y tareas
# crew = Crew(
#     agents=agents,  # Lista de todos los agentes del equipo
#     tasks=[task1, task2, task3, task4],  # Lista de tareas en orden de ejecución
#     verbose=2  # Nivel de detalle en los logs (2 = muy detallado, útil para ver todo el proceso)
# )

# Paso 3: ¡Iniciar el equipo! kickoff() pone a trabajar a todos los agentes
# result = crew.kickoff()

# Paso 4: Mostrar el resultado final
# print(result)

## ⚠️ ADVERTENCIA: Costos de Ejecución

### 💰 Esta aplicación NO es gratuita
* Ejecutar aplicaciones Multi-Agente con GPT-4 es **MÁS CARO** que aplicaciones tradicionales
* **Costo estimado**: Aproximadamente $0.21 USD por ejecución (basado en nuestras pruebas)
* **Por qué es más caro**: 
  - Múltiples agentes ejecutándose
  - Cada agente hace varias llamadas al modelo
  - GPT-4 Turbo es más costoso que GPT-3.5

### 📊 Monitoreo de Costos
* **SIEMPRE revisa el costo total en LangSmith** antes de ejecutar múltiples veces
* Puedes ver el consumo de tokens en tiempo real en el panel de LangSmith

## Ejecutar la Aplicación desde la Terminal

### Comandos para ejecutar:
```bash
cd app
poetry run python3 multiagent.py
```

### ¿Qué hacen estos comandos?
1. `cd app` - Navega al directorio donde está el archivo Python
2. `poetry run python3 multiagent.py` - Ejecuta el script usando el entorno virtual de Poetry

## ⏳ Advertencia Importante: Las Redes Multi-Agente Toman Tiempo

### Sé Paciente
* **Las aplicaciones Multi-Agente NO son instantáneas** - pueden tardar varios minutos
* El proceso puede parecer lento, pero es normal
* Los agentes están "pensando", investigando y colaborando entre sí

### ¿Por qué son lentas?
1. Cada agente hace múltiples llamadas a la API de OpenAI
2. Los agentes se pasan información entre sí
3. El supervisor revisa y puede pedir correcciones

### 🏆 Vale la Pena Esperar
* **Ventaja**: Los resultados de las Redes Multi-Agente suelen ser **MUCHO MEJORES** que los de aplicaciones simples (sin agentes)
* La colaboración entre agentes produce contenido más refinado y de mayor calidad

## Ver los Pasos del Flujo de Trabajo en la Terminal
Una vez que ejecutes la aplicación, verás en la terminal cómo cada agente trabaja paso a paso.
Esto te permite entender exactamente qué está haciendo cada agente en tiempo real.

## 🤖 Ejemplo Sorprendente: ¡Un Agente de IA Tomando Decisiones en Tiempo Real!

### Salida Fascinante del Agente de Búsqueda Online

Observa este increíble output que obtuvimos del agente durante la primera etapa:

> *"Thought: Dado que los intentos de búsqueda iniciales y subsecuentes han llevado a la misma fuente inaccesible de ScienceDirect, y considerando las limitaciones de mis herramientas y la falta de fuentes alternativas proporcionadas por la herramienta de búsqueda, **necesito adaptar mi enfoque para cumplir la tarea sin acceso directo al contenido de la fuente**. Sintetizaré una comprensión general del comportamiento agéntico basándome en la información breve disponible del resultado de búsqueda y el conocimiento general del tema."*

### 🎯 ¿Qué Acabamos de Ver?

**¡Eso fue un Agente de IA razonando y tomando decisiones en tiempo real!** 

El agente:
1. **Detectó un problema** - La fuente no era accesible
2. **Analizó la situación** - Reconoció las limitaciones de sus herramientas
3. **Adaptó su estrategia** - Decidió usar un enfoque alternativo
4. **Tomó acción** - Sintetizó información de lo que tenía disponible

### 🚀 El Futuro es Ahora

¡Imagina lo que las Aplicaciones Multi-Agente LLM podrán hacer muy pronto! Esta capacidad de razonamiento autónomo y adaptación es solo el comienzo.

### 📈 Mejora Continua: Ajusta y Experimenta

Los resultados de la aplicación mejorarán significativamente si ajustas:
* **Los backstories de los agentes** - Dale más contexto y personalidad
* **Las descripciones de las tareas** - Sé más específico sobre qué quieres

**Dejamos esta tarea en tus manos**: Experimenta con diferentes configuraciones y compara los resultados que obtienes con distintos enfoques.

## 📊 Monitoreo con LangSmith: Entendiendo el Rendimiento

### Qué Revisar en LangSmith

1. **Consumo de Tokens**
   - Verás que el proyecto consume MUCHOS tokens
   - Esto explica por qué es más costoso que aplicaciones tradicionales

2. **Monitoreo Individual de Agentes**
   - Cada agente es monitoreado por separado
   - Puedes ver exactamente cuántos tokens consume cada uno

3. **Exploración Detallada**
   - Haz clic en cada agente para ver:
     - **Input**: Qué información recibió el agente
     - **Output**: Qué produjo el agente
     - **Proceso de pensamiento**: Cómo razonó el agente

### 💡 Por Qué Es Útil LangSmith

* **Debugging**: Si algo sale mal, puedes ver exactamente dónde
* **Optimización**: Identifica qué agentes consumen más recursos
* **Aprendizaje**: Entiende cómo "piensan" los agentes de IA